In [2]:
import pandas as pd 
import numpy as np

data_genetics_correlations = pd.read_feather(f"../../all_data/genetics/correlations/correlations.feather").to_dict()

In [3]:
from dash_website.utils.graphs.dendrogram_heatmap import create_dendrogram_heatmap
import plotly.graph_objs as go

correlations = pd.DataFrame(data_genetics_correlations)

table_correlations = correlations.pivot(
    index=["dimension_1", "subdimension_1"],
    columns=["dimension_2", "subdimension_2"],
    values="correlation",
)

customdata_list = []
for customdata_item in [
    "correlation_std",
    "r2_1",
    "r2_std_1",
    "h2_1",
    "h2_std_1",
    "r2_2",
    "r2_std_2",
    "h2_2",
    "h2_std_2",
]:
    customdata_list.append(
        correlations.pivot(
            index=["dimension_1", "subdimension_1"],
            columns=["dimension_2", "subdimension_2"],
            values=customdata_item,
        ).values
    )
stacked_customdata = list(map(list, np.dstack(customdata_list)))

customdata = pd.DataFrame(None, index=table_correlations.index, columns=table_correlations.columns)
customdata[customdata.columns] = stacked_customdata

hovertemplate = "Correlation: %{z:.3f} +- %{customdata[0]:.3f} <br><br>Dimensions 1: %{x} <br>r²: %{customdata[1]:.3f} +- %{customdata[2]:.3f} <br>h²: %{customdata[3]:.3f} +- %{customdata[4]:.3f} <br>Dimensions 2: %{y}<br>r²: %{customdata[5]:.3f} +- %{customdata[6]:.3f}<br>h²: %{customdata[7]:.3f} +- %{customdata[8]:.3f}<br><extra></extra>"


In [6]:
table_correlations

dimension_2                          * *instances01 *instances1.5x  \
subdimension_2                       *            *              *   
dimension_1      subdimension_1                                      
*                *                 NaN          NaN            NaN   
*instances01     *                 NaN          NaN       0.986421   
*instances1.5x   *                 NaN     0.986421            NaN   
*instances23     *                 NaN          NaN            NaN   
Abdomen          *                 NaN          NaN            NaN   
                 Liver             NaN          NaN      -0.197093   
                 Pancreas          NaN          NaN       0.671523   
Arterial         *                 NaN          NaN       1.000000   
                 Carotids          NaN          NaN            NaN   
                 PulseWaveAnalysis NaN          NaN       0.108788   
Biochemistry     *                 NaN          NaN       0.037134   
                 Blood             NaN          NaN       0.080110   
                 Urine             NaN          NaN       0.357914   
BloodCells       *                 NaN          NaN       0.050421   
Brain            *                 NaN          NaN      -0.191328   
                 Cognitive         NaN          NaN      -0.087364   
                 MRI               NaN          NaN      -0.396896   
Eyes             *                 NaN          NaN       0.024334   
                 Fundus            NaN          NaN       0.283063   
                 OCT               NaN          NaN      -0.049587   
Hearing          *                 NaN          NaN       0.191918   
Heart            *                 NaN          NaN       0.310079   
                 ECG               NaN     0.996617       0.451731   
                 MRI               NaN    -1.000000       0.237100   
Lungs            *                 NaN          NaN       0.102739   
Musculoskeletal  *                 NaN          NaN       0.054674   
                 FullBody          NaN          NaN       0.016591   
                 Hips              NaN          NaN      -0.067616   
                 Knees             NaN          NaN       0.098903   
                 Scalars           NaN          NaN       0.068212   
                 Spine             NaN          NaN      -0.371051   
PhysicalActivity *                 NaN     0.986415       0.999015   

dimension_2                        *instances23     Brain                      \
subdimension_2                                *         * Cognitive       MRI   
dimension_1      subdimension_1                                                 
*                *                          NaN       NaN       NaN       NaN   
*instances01     *                          NaN       NaN       NaN       NaN   
*instances1.5x   *                          NaN -0.191328 -0.087364 -0.396896   
*instances23     *                          NaN  0.710316  0.397386  0.768987   
Abdomen          *                     1.000000       NaN       NaN       NaN   
                 Liver                 1.000000       NaN       NaN       NaN   
                 Pancreas              0.966220       NaN       NaN       NaN   
Arterial         *                     0.016989       NaN       NaN       NaN   
                 Carotids             -0.217423       NaN       NaN       NaN   
                 PulseWaveAnalysis     0.357011       NaN       NaN       NaN   
Biochemistry     *                     0.135405       NaN       NaN       NaN   
                 Blood                -0.954358       NaN       NaN       NaN   
                 Urine                      NaN       NaN       NaN       NaN   
BloodCells       *                    -0.316735  0.134277  0.016190  0.099080   
Brain            *                     0.710316       NaN  0.303456  0.957501   
                 Cognitive             0.397386  0.303456       NaN  0.018733   
          

In [7]:
def heatmap_by_clustering(table_correlations, hovertemplate, customdata):
    return create_dendrogram_heatmap(table_correlations, hovertemplate, customdata)

In [8]:
def heatmap_by_sorted_dimensions(sorted_table_correlations, hovertemplate, sorted_customdata):
    heatmap = go.Heatmap(
        x=np.arange(5, 10 * sorted_table_correlations.shape[1] + 5, 10),
        y=np.arange(5, 10 * sorted_table_correlations.shape[1] + 5, 10),
        z=sorted_table_correlations,
        colorscale=BLUE_WHITE_RED,
        customdata=sorted_customdata,
        hovertemplate=hovertemplate,
        zmin=-1,
        zmax=1,
    )

    fig = go.Figure(heatmap)

    fig.update_layout(
        xaxis={
            "tickvals": np.arange(5, 10 * sorted_table_correlations.shape[1] + 5, 10),
            "ticktext": [" - ".join(elem) for elem in sorted_table_correlations.columns.values],
        },
        yaxis={
            "tickvals": np.arange(5, 10 * sorted_table_correlations.shape[0] + 5, 10),
            "ticktext": [" - ".join(elem) for elem in sorted_table_correlations.index.values],
        },
    )

    return fig

In [ ]:
def add_custom_legend_axis(fig, sorted_table_correlations):
    dimensions = (
        sorted_table_correlations.index.to_frame()[["dimension_1", "subdimension_1"]]
        .reset_index(drop=True)
        .rename(columns={"dimension_1": "dimension", "subdimension_1": "subdimension"})
    )
    dimensions["position"] = fig["layout"]["xaxis"]["tickvals"]
    dimensions.set_index(["dimension", "subdimension"], inplace=True)

    lines = []
    annotations = []

    for dimension in dimensions.index.get_level_values("dimension").drop_duplicates():
        dimension_inner_margin = -30
        dimension_outer_margin = -60

        min_position = dimensions.loc[dimension].min()
        max_position = dimensions.loc[dimension].max()

        for first_axis, second_axis in [("x", "y"), ("y", "x")]:
            if first_axis == "x":
                textangle = 90
            else:  # first_axis == "y"
                textangle = 0

            line, annotation = add_line_and_annotation(
                dimension,
                first_axis,
                second_axis,
                min_position,
                max_position,
                dimension_inner_margin,
                dimension_outer_margin,
                textangle,
                10,
            )

            lines.append(line)
            annotations.append(annotation)

            for subdimension in dimensions.loc[dimension].index.get_level_values("subdimension").drop_duplicates():
                subdimension_margin = 0

                submin_position = dimensions.loc[(dimension, subdimension)].min()
                submax_position = dimensions.loc[(dimension, subdimension)].max()

                for first_axis, second_axis in [("x", "y"), ("y", "x")]:
                    if first_axis == "x":
                        textangle = 90
                    else:  # first_axis == "y"
                        textangle = 0

                    line, annotation = add_line_and_annotation(
                        subdimension,
                        first_axis,
                        second_axis,
                        submin_position,
                        submax_position,
                        subdimension_margin,
                        dimension_inner_margin,
                        textangle,
                        8,
                    )

                    lines.append(line)
                    annotations.append(annotation)

    # The final top/right line
    for first_axis, second_axis in [("x", "y"), ("y", "x")]:
        line, _ = add_line_and_annotation(
            dimension,
            first_axis,
            second_axis,
            min_position,
            max_position,
            0,
            dimension_outer_margin,
            0,
            10,
            final=True,
        )

        lines.append(line)

    fig["layout"]["shapes"] = lines
    fig["layout"]["annotations"] = annotations
    fig.update_layout(yaxis={"showticklabels": False}, xaxis={"showticklabels": False})

    return fig
